In [1]:
from imutils.paths import list_files
from datetime import datetime
import traceback
import requests
import os, re
import time
import json

In [2]:
IS_JUPYTER = True
ROOT_PATH  = '/home/jovyan/project' if IS_JUPYTER else '/config/workspace/project'
DATA_PATH  = f'{ROOT_PATH}/DoveNest/steam/jsons'

JSON_PATH        = f'{ROOT_PATH}/utils/keys'
JSON_BACKUP_PATH = f'{ROOT_PATH}/BACKUPS/keys'

In [3]:
unix2datetime = lambda unixtime: str(datetime.fromtimestamp(unixtime))
load_json = lambda json_path: json.loads(open(json_path, 'r').read())
save_json = lambda data, json_path: json.dump(data, open(json_path, 'w'))

def repair_keys(json_path):
    try: keys = load_json(f'{json_path}/keys.json')
        
    except Exception as e:
        print(f'[ERR.K-002] json 파일이 깨져 열 수 없습니다. {e}')
        text = open(f'{JSON_BACKUP_PATH}/keys.txt', 'r').read().split('\n')
        keys = {platform : key 
                for platform, key in zip(['youtube', 'steam'], text)}
        
        save_paths = [JSON_PATH, JSON_BACKUP_PATH]
        
        print(save_paths)
        for save_path in save_paths:
            
            print(save_path)
            os.makedirs(save_path, exist_ok = True)
            save_json(keys, f'{save_path}/keys.json')
        
    finally: return keys

def return_or_print(response):
    
    if response.status_code == 200: return response.json()
    else: print(f'no response data with code : {response.status_code}')

    
get_api  = lambda url: return_or_print(requests.get(url))

def get_info(appid):
    
    response = requests.get(f'https://store.steampowered.com/api/appdetails?appids={appid}&l=korean')
    return return_or_print(response)

In [4]:
if os.path.isfile(f'{JSON_PATH}/keys.json'):
    key = repair_keys(JSON_PATH)['steam']
        
else:
    print(f'[ERR.K-001] json 파일이 존재하지 않아 백업 데이터를 로딩합니다.')
    key = repair_keys(JSON_BACKUP_PATH)['steam']

In [5]:

def get_user_library(key, games):
    
    library_url = f'''https://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={key}&steamid={user_id}&format=json'''
    games = get_api(library_url)['response']['games']
    
    user_datas = []
    for game in games:
        try:
            last_play_time = unix2datetime(game['rtime_last_played'])
            
            info = [game['appid'], game['playtime_forever'], str(last_play_time)]
            user_datas.append(info)
            
        except Exception as e:
            print(traceback.format_exc(), e)
            
    return user_datas


In [6]:
user_id   = 76561198032052956
game_info = get_user_library(key, user_id)
game_info = sorted(game_info, key = lambda x: x[1], reverse = True)

In [7]:
## '23.01.06 10:41분 기준 152,735개
len(requests.get('https://api.steampowered.com/ISteamApps/GetAppList/v2/').json()['applist']['apps'])

152736

In [8]:
def get_json(appid):
    
    json_path = f'{DATA_PATH}/{appid}/{appid}.json'
    if os.path.isfile(json_path):
        json_data = json.loads(open(json_path, 'r').read())
        
        return json_data

    else:
        print(f'[ERR-J0001] <{appid}> json 파일이 존재하지 않습니다.')
        return {}

In [9]:
for game in game_info:
    
    appid, _, _ = game
    
    try:
        json_data           = get_json(appid)
        
        if 'price_overview' in json_data.keys():
            discount_percentage = json_data['price_overview']['discount_percent']
            if json_data == 0: print(discount_percentage, '\n')
            
        else:
            print(f'[ERR-K0001] <{appid}> 가격정보를 제공하지 않습니다.\n')
        
    except Exception as e:
        print(f'[ERR]<{appid}> {e}\n')

[ERR-J0001] <24840> json 파일이 존재하지 않습니다.
[ERR-K0001] <24840> 가격정보를 제공하지 않습니다.

[ERR-J0001] <21970> json 파일이 존재하지 않습니다.
[ERR-K0001] <21970> 가격정보를 제공하지 않습니다.

[ERR-K0001] <342370> 가격정보를 제공하지 않습니다.

[ERR-J0001] <287450> json 파일이 존재하지 않습니다.
[ERR-K0001] <287450> 가격정보를 제공하지 않습니다.

[ERR-K0001] <730> 가격정보를 제공하지 않습니다.

[ERR-K0001] <63380> 가격정보를 제공하지 않습니다.

[ERR-K0001] <317600> 가격정보를 제공하지 않습니다.

[ERR-K0001] <236390> 가격정보를 제공하지 않습니다.

[ERR-K0001] <314420> 가격정보를 제공하지 않습니다.

[ERR-K0001] <208480> 가격정보를 제공하지 않습니다.

[ERR-K0001] <681590> 가격정보를 제공하지 않습니다.

[ERR-K0001] <21910> 가격정보를 제공하지 않습니다.

[ERR-J0001] <33910> json 파일이 존재하지 않습니다.
[ERR-K0001] <33910> 가격정보를 제공하지 않습니다.

[ERR-K0001] <44350> 가격정보를 제공하지 않습니다.

[ERR-K0001] <44340> 가격정보를 제공하지 않습니다.

[ERR-K0001] <230410> 가격정보를 제공하지 않습니다.

[ERR-K0001] <304930> 가격정보를 제공하지 않습니다.

[ERR-K0001] <21760> 가격정보를 제공하지 않습니다.

[ERR-K0001] <335330> 가격정보를 제공하지 않습니다.

[ERR-K0001] <1097150> 가격정보를 제공하지 않습니다.

[ERR-J0001] <304050> json 파일이 존재하지 않습니다.
[ERR-K0001] <304050> 가격정보를 제

In [10]:
get_json(105450)['developers']

['Ensemble Studios']

In [11]:
from bs4 import BeautifulSoup as bs

In [20]:
req  = requests.get('https://dove-nest.duckdns.org:333/exp/').text
soup = bs(req, 'html.parser') 
soup.select('li.is_on > div')[0]['id']

'tab1'